### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7297,2023-11-25,Brasil Nbb,18:00,Mogi,R10 Score Vasco da Gama,2.88,1.38,144.5,1.84,1.86,5.5,2.05,1.69,pAmKN1Xq,0.347222,0.724638,0.543478,0.537634,0.071860,0.6,0.0,NaN,NaN,290.554,135.743466,0.467188,NaN,NaN,NaN,265.508,0.0,478.16,0.00,1.0,0.0,0.0,0.0,0.497963,0.007644,0.136128,-3.07,-0.614,-3.061889,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7298,2023-11-25,Brasil Nbb,18:00,Paulistano,Minas,2.34,1.55,148.5,1.81,1.86,2.5,2.10,1.65,vXnOMLnj,0.427350,0.645161,0.552486,0.537634,0.072512,0.0,0.0,NaN,NaN,128.346,27.596106,0.215013,114.774,20.993651,0.182913,0.000,0.0,101.12,136.95,0.0,0.0,0.0,0.0,0.287205,0.019267,0.169706,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7299,2023-11-25,Espanha Acb,14:00,Obradoiro CAB,Granada,1.37,3.12,164.5,1.80,1.94,-8.5,2.10,1.65,dtRhBIHd,0.729927,0.320513,0.555556,0.515464,0.050440,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,257.04,333.00,0.0,0.0,0.0,0.0,0.551197,0.052938,0.169706,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7300,2023-11-25,Espanha Acb,14:00,Palencia,Valencia,3.12,1.37,153.5,1.88,1.90,5.5,2.01,1.76,IN9E0sMR,0.320513,0.729927,0.531915,0.526316,0.050440,0.0,0.0,NaN,NaN,NaN,NaN,NaN,137.236,39.355971,0.286776,0.000,0.0,164.34,182.04,0.0,0.0,0.0,0.0,0.551197,0.007483,0.093781,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7301,2023-11-25,Espanha Acb,16:45,Bilbao,Unicaja,2.90,1.42,163.5,1.82,1.96,4.5,2.03,1.73,0EiZJALd,0.344828,0.704225,0.549451,0.510204,0.049053,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,371.28,451.44,1.0,0.0,0.0,0.0,0.484499,0.052378,0.112836,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7562,2023-11-25,Eua Ncaa,21:00,Richmond,Queens Royals,1.13,6.40,142.5,1.80,1.95,-13.5,2.10,1.67,2eJx0cdq,0.884956,0.156250,0.555556,0.512821,0.041206,0.0,0.0,NaN,NaN,158.304,77.981199,0.492604,NaN,NaN,NaN,0.000,0.0,288.51,226.44,0.0,0.0,0.0,0.0,0.989762,0.056569,0.161303,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7563,2023-11-25,Eua Ncaa,21:00,St. Johns,Holy Cross,1.02,14.50,153.5,1.80,1.95,-22.5,2.00,1.71,4jCGsjO2,0.980392,0.068966,0.555556,0.512821,0.049358,0.0,0.0,NaN,NaN,NaN,NaN,NaN,375.070,403.945642,1.076987,0.000,0.0,131.40,126.54,0.0,0.0,0.0,0.0,1.228325,0.056569,0.110545,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7564,2023-11-25,Eua Ncaa,21:00,Wichita State,Norfolk State,1.07,9.80,142.5,1.91,1.91,-16.5,2.10,1.67,YyyMm1KI,0.934579,0.102041,0.523560,0.523560,0.036620,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,97.27,167.68,0.0,0.0,0.0,0.0,1.135794,0.000000,0.161303,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7565,2023-11-25,Eua Ncaa,22:00,Grand Canyon,North Dakota St.,1.08,9.30,142.5,1.91,1.91,-15.5,2.10,1.67,jPJGToG7,0.925926,0.107527,0.523560,0.523560,0.033453,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,98.60,309.66,0.0,0.0,0.0,0.0,1.119926,0.000000,0.161303,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
7,00:30,Eua Nba,Los Angeles Clippers,New Orleans Pelicans,224.5,1.82,1.0,Over
